In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import shutil
import json
import re

pd.set_option('display.max_colwidth', None)

In [16]:
preds_dict = {}
for i in range(5):
    path = '../../../ceph_data/output/bart-AAE-v2-only-dot-direct-cola-au-full-mask-gen/{}/aee.preds'.format(i)
    path2 = '../../../ceph_data/intermediate/bart-AAE-v2-only-dot-direct-cola-au-full-mask-gen/{}/test.target'.format(i)
    with open(path, encoding='utf-8') as h:
        preds = h.readlines()
    with open(path2, encoding='utf-8') as f:
        gts = f.readlines()
    for pred, gt in zip(preds,gts):
        preds_dict[gt.replace('\n','')]=pred.replace('\n','')

In [17]:
df_aae_full = pd.read_csv('../../../ceph_data/input/UKP-InsufficientArguments_v1.0/data-tokenized.tsv', sep='\t', index_col=False, encoding='latin-1')
df_aae = pd.read_json('../../../ceph_data/intermediate/corpus-ukp-argument-annotated-essays-v2/aae3.json')
df_aae_invalid = pd.read_json('../../../ceph_data/intermediate/corpus-ukp-argument-annotated-essays-v2/aae3_invalid.json')
df_aae_fixed = pd.read_json('../../../ceph_data/intermediate/corpus-ukp-argument-annotated-essays-v2/aae3_fixed.json')
df_split = pd.read_csv('../../../ceph_data/input/UKP-InsufficientArguments_v1.0/data-splitting.tsv', sep='\t', names=['index']+[str(i) for i in range(100)], index_col=False)

df_aae_full['index'] = df_aae_full.apply(lambda x: 'essay{}_{}'.format(str(x['ESSAY']).zfill(3), x['ARGUMENT']), axis=1)
df_aae_full['local_sufficency'] = df_aae_full['ANNOTATION'].apply(lambda x: 0 if x == 'insufficient' else 1)


In [18]:
df_aae_full_mask = pd.read_csv('df_aae_full_mask.csv')

In [19]:
df_aae_full_mask.head()

,Unnamed: 0,index,local_sufficency,TEXT,TEXT_MASK
0,0,essay001_1,1.0,"through cooperation, children can learn about interpersonal skills which are significant in the future life of all students","First of all, <mask>. What we acquired from team work is not only how to achieve the same goal with others but more importantly, how to get along with others. During the process of cooperation, children can learn about how to listen to opinions of others, how to communicate with others, how to think comprehensively, and even how to compromise with other team members when conflicts occurred. All of these skills help them to get on well with other people and will benefit them for the whole life."
1,1,essay001_2,0.0,competition makes the society more effective,"On the other hand, the significance of competition is that how to become more excellence to gain the victory. Hence it is always said that <mask>. However, when we consider about the question that how to win the game, we always find that we need the cooperation. The greater our goal is, the more competition we need. Take Olympic games which is a form of competition for instance, it is hard to imagine how an athlete could win the game without the training of his or her coach, and the help of other professional staffs such as the people who take care of his diet, and those who are in charge of the medical care. The winner is the athlete but the success belongs to the whole team. Therefore <unk>."
2,2,essay002_1,0.0,sustaining the cultural values of immigrants is paramount essential,"Firstly, maintaining one’s cultural identity is a key important rule to help individuals emerge in the new multicultural environments. Take Australia for example, immigrants from varieties of nations have a day called multicultural day where people from each country prepare their food and traditional activities for displaying in the public venues. Many Australians come this day to enjoy the shows, learn about the cultures and admire the diverse values. These feedbacks, in turn, help raise one’s pride of their cultures and help people understand each other more. Thus this makes it clear that <mask>."
3,3,essay002_2,1.0,keeping the cultural traditions in the destination countries is tremendous important,"Secondly, it is crucial to keep one’s identity for they need a connection back to their country as well as teach their children their value of origin. For instance, children immigrated to a new country will face social troubles in school with new friends. In this new environment, parent should find friends coming from their same country so that they can socialize in a very familiar manner as feeling being home. Fail to create this familiarity makes them felt isolated, in the extreme can lead to social disorder like autism. Hence, it is clear that <mask>."
4,4,essay003_1,0.0,tourism has clearly improved lives in the tourist country,"Firstly, international tourism promotes many aspects of the destination country’s economy in order to serve various demands of tourists. Take Cambodia for example, a large number of visitors coming to visit the Angkowat ancient temple need services like restaurants, hotels, souvenir shops and other stores. These demands trigger related business in the surrounding settings which in turn create many jobs for local people improve infrastructure and living standard. Therefore <mask>."


In [20]:
only_claims = list(set(df_aae_full['index']) - set(df_aae['index']) - set(df_aae_invalid['index']))
df_aae_full_only_claims = df_aae_full[df_aae_full['index'].isin(only_claims)]

In [21]:
df_aae.columns

Index(['index', 'topic', 'para_text', 'premises', 'conclusion',
       'local_sufficency', 'au', 'major_claim'],
      dtype='object')

In [22]:
df_aae_full_only_claims.columns

Index(['ESSAY', 'ARGUMENT', 'TEXT', 'ANNOTATION', 'index', 'local_sufficency'], dtype='object')

In [23]:
df_aae_full_only_claims.head()

,ESSAY,ARGUMENT,TEXT,ANNOTATION,index,local_sufficency
46,20,3,"On the other hand , I believe that governments should restrict the use of products that cause air pollution for both individuals and companies . Governments also should use an amount of money from their budgets to clean up the air , instead of putting all the responsibility to companies and private individuals .",NaN,essay020_3,1
528,205,1,"Firstly , a liberal policy is very feasible . Checking for attendance requires a lot of bureaucracy . Especially for larger classes it is impossible to check for everybody to attend . So an optional attendance saves time and money .",NaN,essay205_1,1
826,315,2,"Another thing that put big cities in front of small towns is the facilitates and services it contains which support and improve the quality of its inhabitants ' lives , hospitals , advanced medical facilities can satisfy one ' s need of medical safety . Cultural events such as exhibitions and theatre plays can enrich one ' s and widen his horizons , things which aren ' t offered by small towns .",NaN,essay315_2,1
876,335,1,"Firstly , connecting people by email is easy and fast . In addition , World Wide Web offers humanity to access to information , which they want to know for less than 10 seconds . These are two of the benefits , why IT is useful .",NaN,essay335_1,1


In [24]:
df_aae_invalid.columns

Index(['index', 'topic', 'para_text', 'premises', 'conclusion',
       'local_sufficency', 'au', 'major_claim', 'removal_reason'],
      dtype='object')

In [25]:
preds_dict = {}
for i in range(5):
    path = './masterthesis/ceph_data/output/bart-AAE-v2-only-dot-direct-cola-au-full-mask-gen/{}/aee.preds'.format(i)
    path2 = './masterthesis/ceph_data/intermediate/bart-AAE-v2-only-dot-direct-cola-au-full-mask-gen/{}/test.target'.format(i)
    with open(path, encoding='utf-8') as h:
        preds = h.readlines()
    with open(path2, encoding='utf-8') as f:
        gts = f.readlines()
    for pred, gt in zip(preds,gts):
        preds_dict[gt.replace('\n','')]=pred.replace('\n','')

In [27]:
preds_dict

{'sustaining the cultural values of immigrants is paramount essential': " keeping one’s cultural identity is the best way to maintain one's cultural identity",
 'one who has studied and lived overseas will become more eligible for the job than his/her peers': ' this study will give the student an edge in the job market',
 'the number of people reading newspapers may continue falling sharply, possibly leading to the close-downs of many in the coming time': ' another reason why the internet is a waste of money is the lack of access to newspapers',
 "instead of interfering children's creativity, technology actually encourages children to learn and to create": ' technology inspires children to create new things',
 'students at schools and universities still learn much more from lessons with teachers': ' teachers are the best teachers for their students',
 'it is easy to point out many benefits users can get from email and text messaging, which answers to the question of their great popular

In [28]:
conclusions_dict = {}
for i, row in df_aae.iterrows():
    if row['index'] not in conclusions_dict:
        conclusions_dict[row['index']] = row['conclusion']
for i, row in df_aae_invalid.iterrows():
    if row['index'] not in conclusions_dict:
        conclusions_dict[row['index']] = row['conclusion']
    else:
        conclusions_dict[row['index']] = conclusions_dict[row['index']] + '. '+row['conclusion']
for i, row in df_aae_full_only_claims.iterrows():
    conclusions_dict[row['index']] = row['TEXT']

In [29]:
for index, conclusion in conclusions_dict.items():
    for pred, gt in zip(preds,gts):
        if gt.replace('\n','') in conclusion:
            conclusions_dict[index] += ' </s> ' + pred.replace('\n','')

In [30]:
df_aae_full['TEXT'] = df_aae_full['index'].apply(lambda x: conclusions_dict[x])

In [35]:
df_aae_full_mask['TEXT2'] = df_aae_full_mask['TEXT'].apply(lambda x: preds_dict[x])

In [36]:
df_aae_full_mask.sample(20)

,Unnamed: 0,index,local_sufficency,TEXT,TEXT_MASK,TEXT2
992,992,essay388_1,1.0,if there is gun control somehow crime has to decrease,"First of all, it seems to be true that <mask> because fewer guns available mean less crime. However, this is not as simple as it sounds.","first of all, it seems to be true that fewer guns will reduce crime"
1039,30,essay086_3,1.0,"they can exchange their knowledge, they can learn different cultures","Finally, in big cities children access to media easily, so <unk>. Via internet they can have connection with the other children all around the world, therefor <mask>.","in big cities, children can have access to media easily, there is no doubt about it"
245,245,essay098_1,1.0,"they are likely to understand their parents' hardship, to respect values of money and to form a good habit of saving","First of all, children can learn about the hardship of earning money. When they take some kind of paid work such as looking after babies, doing housework and keeping shops, they will have some life experience and skills which make them mature and can deal with many different situations in real life. Therefore, <mask>.","first of all, children should learn how to earn money"
751,751,essay286_3,1.0,experiences are also change to plan for my future life,"Lastly, <mask>. For example, I read about saving money for my retire is a good plan for my life. Consequently, I start changing myself from spending many thing that I want to saving money for my entire life. This experience change me to concern about making plan for my future life.",saving money for my retirement is a good plan for my life
769,769,essay293_2,1.0,"less misunderstanding leads to less conflicts and fights, and enhance the communication amongst the team members","Secondly, work in person can indirectly improve the quality and the efficiency of the project because there will be less conflicts or fight amongst the team members. Writing an email for an informal matter will make that matter looks official, therefore people who receives the email may feel nervous or maybe feel offended by the words in writing. While if the same words are expressed in talking, the counterparty will feel more relaxed since their response will not be recorded and they will not misunderstand the informal discussion to an official claim. Therefore, <mask>.",working in person can improve the quality and efficiency of the project
433,433,essay172_2,1.0,an apartment is more expensive,"Second, from an economic point of view, people often argue that <mask>. This is only partially true, you could rent the extra rooms in the apartment to other people and rent would be a lot cheaper. Transportation is another issue, you have to go to and return from the university everyday. You could choose to use public transportation, a cheaper option, or to have a car. The car is an expensive option, but also offers freedom of schedule and destination.","second, living in an apartment is more expensive than living in a house"
1009,1009,essay394_2,0.0,poor handwriting can have a negative impact on students' school performance,"Secondly, parents and schools should give priority on directing their kids about handwriting as <mask>. The fact is that kids who are bad at handwriting meet difficulties in taking notes and catching up lessons. It resulted in the missing of information and the lack of details during their works. Moreover, the increasing taking advantage of advanced equipment could make students be lazier and more subjective in their writing skills. For example, with the assistance of automatic grammar and spelling checking in office word, learners are now no need for noticing much in their mistakes.","with the advent of technology, students are more likely to make mistakes in their writing"
191,191,essay077_2,1.0,"zoos afford a platform to education teenagers, helping to establish eco-friendly behavior towards animals","Additionally, <mask>. In the zoos, even though wild animals are caged, t

In [37]:
df_aae_full_mask['REPLACED_TEXT'] = df_aae_full_mask[['TEXT','TEXT2']].apply(lambda x: x[0]+' </s> '+x[1],axis=1)

In [38]:
df_aae_full_mask.sample(20)

,Unnamed: 0,index,local_sufficency,TEXT,TEXT_MASK,TEXT2,REPLACED_TEXT
782,782,essay298_1,0.0,"students can learn lots of things from part time job, before starting their real job","The main reason is that <mask>. Students can learn about time management, money management and convince people by their words. For example, when I was in my high school I worked as a sales-man in a shop for a year. As i had to spend some of my time in my job, therefore, i had to manage my time efficiently. I had to manage my boss as well as had to prepare my homework. This taught me how to manage time efficiently and do any job quickly and effectively. Moreover, I learn money management. When my boss was out of the shop, i had to manage the accounts section of the shop. From this i learned, how to manage money correctly. This experience taught me, doing part time job can help students to learn many things.",doing part time job can help students to learn many things,"students can learn lots of things from part time job, before starting their real job </s> doing part time job can help students to learn many things"
234,234,essay093_1,1.0,"it is inhumane to watch one's fellowman struggle against famine and poverty without doing anything to help, especially when the onlooker takes no effort to do so","In terms of ethics, <mask>. Compassion, a virtue preached in almost all religions from Buddhism to Mohammedanism, is what distinguishes us human beings from animals.","In terms of religion, compassion is one of the most important values in human life","it is inhumane to watch one's fellowman struggle against famine and poverty without doing anything to help, especially when the onlooker takes no effort to do so </s> In terms of religion, compassion is one of the most important values in human life"
638,638,essay246_1,0.0,making quick decisions leads to probable bad results,"For one thing, making a quick decision means that it's not very thought out, and that it hasn't been checked from all its aspects. Consequently, <mask>. For example: if someone decided to quit his job without really thinking about the alternative, or a plan B, he could end up being unemployed, because of his speedy decision.",making a quick decision is not a good idea,making quick decisions leads to probable bad results </s> making a quick decision is not a good idea
794,794,essay302_3,0.0,Working hard and saving money for later on investments is a great way of getting a satisfying feeling and to reward yourself for something you have accomplished,"Finally, with savings you can afford some luxurious goods and fulfill your dreams. <mask>. For instance, I made a month-long backpack trip through Southeast Asia last summer with my girlfriend. Without our savings we could not have pursued our long-time dream.","with savings, you can pursue your long-time dream","Working hard and saving money for later on investments is a great way of getting a satisfying feeling and to reward yourself for something you have accomplished </s> with savings, you can pursue your long-time dream"
745,745,essay284_3,1.0,we should show people the truth in the stories,"In fact, those good endings sometimes are helpful. Some people may be encouraged to do good things. But like I said, this kind of behavior won’t last long, because someday they will realize the truth. So I suggest <mask>. And if they can, they will be good people no matter how the story ends.","in order of life, people have to learn to be responsible for themselves","we should show people the truth in the stories </s> in order of life, people have to learn to be responsible for themselves"
129,129,essay053_2,1.0,many farmers and owners exploit animals beyond their limits,"On the other hand, <mask>. For instance, cows are often injected by Oxytocin, a hormone which is used to forcibly increase lactation. This in turn, causes lots of discomfort and pain to the cattle. Furthermore, thousand of animals undergo painful suffering or death as a result of sci

In [41]:
df_aae_full_mask['local_sufficency2'] = df_aae_full_mask['local_sufficency'].apply(lambda x: int(x))

In [42]:
df_aae_full_mask.sample(20)

,Unnamed: 0,index,local_sufficency,TEXT,TEXT_MASK,TEXT2,REPLACED_TEXT,placeholder1,placeholder2,placeholder3,split,local_sufficency2
34,34,essay015_2,1.0,"The popularity of email and text messaging, as a result, leads to the losing of written language's position","<mask>. People tend to use slangs and abbreviated words to shorten their writing and save more time in communicating for other activities. The outcome is that language in writing is not in its original form. Even in some cases, people compose a message with only emoticons (the expressions by face created by service providers), which means there is no room for writing or typing texts. Moreover, it is more dangerous that youngsters sometimes use such new languages in their studying, writing assignments. If that tendency continues at a rapid pace, written language may face up to a real problem about its stand.",the use of slangs and abbreviated words in writing is not a good idea,"The popularity of email and text messaging, as a result, leads to the losing of written language's position </s> the use of slangs and abbreviated words in writing is not a good idea",34,34,34,TEST,1
498,498,essay194_1,1.0,Many scientific studies have revealed the detriment that zoos have caused to animals,"<mask>. Firstly, like we, human beings, animals are delivered to this world with genetic instincts. Skills which help them live and survive in their residing areas. When we captivate animals and put them to live a style of life that they are unfamiliar with, they begin to loose those in-born instincts. Hence, when they are released or in the worst case, when they escape, they act disorientated and this contributes to the animal mortality rate.",animals have in-born skills which help them live and survive in their residing areas,Many scientific studies have revealed the detriment that zoos have caused to animals </s> animals have in-born skills which help them live and survive in their residing areas,498,498,498,TRAIN,1
1051,65,essay169_3,0.0,professors could teach students through research and projects,"Nevertheless, <mask>. Some students enjoy taking part in some research. Through to use the equipment and invent their own work, they may get more experience in the process of research. For instance, electronic students always invent new machines such as robots, MP3 and remote control car. They can manage the electric theory in practical skills. And some students will be rewarded by investing new inventions. However, only under specific circumstance, when students master all of the systematic knowledge, can they accomplish research. This is requires professors provide high quality courses. In other word, <unk>.",it is possible for students to do some research,professors could teach students through research and projects </s> it is possible for students to do some research,1051,1051,1051,TRAIN,0
128,128,essay053_1,1.0,"Human being has driven many animals for their personal use such as, cows and other milch animals for yielding milk from them","<mask>. Likewise, other animals such as, horses, cattles and donkeys for farming, agriculture and as a mode of transportation. Many people keep pets like dogs and cats, trained them as service dogs and snuffing dogs which are very helpful to police force and even to some blind peoples. They are kept for security purposes in houses.",many people keep animals as pets,"Human being has driven many animals for their personal use such as, cows and other milch animals for yielding milk from them </s> many people keep animals as pets",128,128,128,DEV,1
200,200,essay080_2,1.0,zoos provide the public with good entertainment and also help to ensure the balance of the ecology,"On the other hand, the advocators believe that <mask>. Many people, especially young children enjoy going to zoos and see their favorite animals, cute baby-born animals and the animals that are from the different corners of the world. Zoos provide educational information about the lives of dif

In [43]:
df_aae_full_mask['placeholder1'] = np.arange(len(df_aae_full_mask))
df_aae_full_mask['placeholder2'] = np.arange(len(df_aae_full_mask))
df_aae_full_mask['placeholder3'] = np.arange(len(df_aae_full_mask))

path = '../../../ceph_data/intermediate/bert-AAE-v2-only-dot-direct-cola-conclusion-gen'
if not os.path.exists(path):
    os.mkdir(path)
else:
    shutil.rmtree(path)
    os.mkdir(path)
    
path2 = '../../../ceph_data/output/bert-AAE-v2-only-dot-direct-cola-conclusion-gen'
if not os.path.exists(path2):
    os.mkdir(path2)
else:
    shutil.rmtree(path2)
    os.mkdir(path2)

for i in range(100):
    if not os.path.exists(path+'/{}'.format(i)):
        os.mkdir(path+'/{}'.format(i))
    else:
        shutil.rmtree(path+'/{}'.format(i))
        os.mkdir(path+'/{}'.format(i))
        
    if not os.path.exists(path2+'/{}'.format(i)):
        os.mkdir(path2+'/{}'.format(i))
    else:
        shutil.rmtree(path2+'/{}'.format(i))
        os.mkdir(path2+'/{}'.format(i))
    
    split_dict = dict(zip(df_split['index'], df_split[str(i)]))
    df_aae_full_mask['split'] = df_aae_full_mask['index'].apply(lambda x: split_dict[x])
    df_aae_full_mask[df_aae_full_mask['split']=='TRAIN'][['local_sufficency2','placeholder1','placeholder2','TEXT2','placeholder3']].to_csv('../../../ceph_data/intermediate/bert-AAE-v2-only-dot-direct-cola-conclusion-gen/'+str(i)+'/train.tsv',sep='\t', index=False)
    df_aae_full_mask[df_aae_full_mask['split']=='DEV'][['local_sufficency2','placeholder1','placeholder2','TEXT2','placeholder3']].to_csv('../../../ceph_data/intermediate/bert-AAE-v2-only-dot-direct-cola-conclusions-gen/'+str(i)+'/dev.tsv',sep='\t', index=False)
    df_aae_full_mask[df_aae_full_mask['split']=='TEST'][['local_sufficency2','placeholder1','placeholder2','TEXT2','placeholder3']].to_csv('../../../ceph_data/intermediate/bert-AAE-v2-only-dot-direct-cola-conclusions-gen/'+str(i)+'/test.tsv',sep='\t', index=False)
